In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
sentences = ["Julia kocha Marka", "Marek kocha Annę", "Anna i Julia kochają Marka", "Dzisiaj świeci słońce"]

In [3]:
cv = CountVectorizer()
wm = cv.fit_transform(sentences)

In [4]:
pd.DataFrame(wm.toarray(), columns=cv.get_feature_names_out())

,anna,annę,dzisiaj,julia,kocha,kochają,marek,marka,słońce,świeci
0,0,0,0,1,1,0,0,1,0,0
1,0,1,0,0,1,0,1,0,0,0
2,1,0,0,1,0,1,0,1,0,0
3,0,0,1,0,0,0,0,0,1,1


In [6]:
pd.DataFrame( cosine_similarity(wm) )

,0,1,2,3
0,1.000000,0.333333,0.57735,0.0
1,0.333333,1.000000,0.00000,0.0
2,0.577350,0.000000,1.00000,0.0
3,0.000000,0.000000,0.00000,1.0


In [8]:
df = pd.read_csv("http://51.91.120.89/extras/movie_dataset.csv")
df.head(1)

index     budget                                    genres  \
0      0  237000000  Action Adventure Fantasy Science Fiction   

                      homepage     id  \
0  http://www.avatarmovie.com/  19995   

                                            keywords original_language  \
0  culture clash future space war space colony so...                en   

  original_title                                           overview  \
0         Avatar  In the 22nd century, a paraplegic Marine is di...   

   popularity  ... runtime                                   spoken_languages  \
0  150.437577  ...   162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...   

     status                      tagline   title vote_average vote_count  \
0  Released  Enter the World of Pandora.  Avatar          7.2      11800   

                                                cast  \
0  Sam Worthington Zoe Saldana Sigourney Weaver S...   

                                                crew       director  
0  [{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...  James Cameron  

[1 rows x 24 columns]

In [9]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [10]:
df.iloc[0]

index                                                                   0
budget                                                          237000000
genres                           Action Adventure Fantasy Science Fiction
homepage                                      http://www.avatarmovie.com/
id                                                                  19995
keywords                culture clash future space war space colony so...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                               

In [11]:
# wybór cech na potrzeby rekomendacji
features = ["genres","keywords","title","cast","director"]

In [13]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [14]:
for col in features:
    df[col].fillna("", inplace=True)

In [15]:
# tworzenie kolumny na potrzeby rekomendacji
df["recommend"] = df.genres+" "+df.keywords+" "+df.title+" "+df.cast+" "+df.director

In [16]:
df.recommend.iloc[123]

'Adventure Action Thriller Science Fiction saving the world artificial intelligence man vs machine flying philosophy The Matrix Revolutions Keanu Reeves Laurence Fishburne Carrie-Anne Moss Hugo Weaving Mary Alice Lilly Wachowski'

In [17]:
cv = CountVectorizer()
wm = cv.fit_transform(df.recommend)
wm.shape

(4803, 17482)

In [18]:
# obliczanie podob. cosinusowego
sim = cosine_similarity(wm)
sim.shape

(4803, 4803)

In [20]:
df.recommend.iloc[126]

'Action Adventure Fantasy marvel comic superhero based on comic book hostile takeover norse mythology Thor: The Dark World Chris Hemsworth Natalie Portman Tom Hiddleston Anthony Hopkins Christopher Eccleston Alan Taylor'

In [21]:
sim[123,126]

0.15386436372416593

In [30]:
def my_sort(x):
    return x[1]
    
sorted( enumerate(sim[123]) , reverse=True, key=lambda x:x[1])[1:11]

[(634, 0.8581163303210326),
 (125, 0.8238858408710998),
 (93, 0.44781107551989896),
 (3439, 0.39477101697586137),
 (108, 0.3555795011093798),
 (212, 0.33850160019316505),
 (505, 0.33850160019316505),
 (43, 0.335012605086404),
 (95, 0.3289758474798845),
 (4401, 0.3289758474798845)]

In [32]:
def get_title_by_index(ix):
    return df.iloc[ix].title

In [41]:
get_title_by_index(1)

"Pirates of the Caribbean: At World's End"

In [43]:
result = sorted( enumerate(sim[1]) , reverse=True, key=lambda x:x[1])[1:11]
for mi, sim_ in result:
    print(get_title_by_index(mi), round(sim_*100,0) , sep=" - ")

Pirates of the Caribbean: The Curse of the Black Pearl - 63.0
Pirates of the Caribbean: Dead Man's Chest - 61.0
The Lord of the Rings: The Fellowship of the Ring - 31.0
Seeking a Friend for the End of the World - 31.0
The Lord of the Rings: The Return of the King - 30.0
Pirates of the Caribbean: On Stranger Tides - 30.0
Bedazzled - 29.0
The Island - 28.0
Kingdom of Heaven - 26.0
The Lord of the Rings: The Two Towers - 26.0
